safety-inspectorAI/

*   ├── app.py                 # Main Gradio application
├── requirements.txt       # Python dependencies
├── train_autoencoder.py   # Unsupervised training
├── detect_anomaly.py      # Anomaly detection logic
├── detect_helmet.py       # YOLOv8 helmet detection
├── models/
│   ├── autoencoder.pth    # Trained autoencoder model
│   └── yolo_helmet.pt     # YOLOv8 helmet detector
└── README.md





In [ ]:
%%writefile Readme.md

---
title: AI Safety Inspector

colorFrom: blue
colorTo: red
sdk: gradio
sdk_version: 4.29.0
app_file: app.py
pinned: false
license: mit
---

#  AI Safety Inspector

Detects safety hazards like missing helmets and danger zones using:
- Unsupervised Learning (Autoencoder)
- YOLOv8 Object Detection

##  Features
-  Anomaly detection without labels
-  Helmet detection with bounding boxes

##  How to Use
1. Upload a construction site image
2. Click "Analyze" to detect hazards
3. Use "Helmet Detection" tab for precise bounding boxes

Built with PyTorch, YOLOv8, and Gradio.

Overwriting Readme.md


In [ ]:
!mkdir -p ai-safety-inspector
%cd ai-safety-inspector

/content/ai-safety-inspector


In [ ]:
%%writefile requirements.txt
torch
torchvision
numpy
scikit-learn
gradio
diffusers
transformers
accelerate
safetensors
Pillow
matplotlib
ultralytics

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
%%writefile train_autoencoder.py

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import os
import numpy as np
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Feature Extractor (ResNet)
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval().to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class ImageDataset(Dataset):
    def __init__(self, img_dir):
        self.img_dir = img_dir
        self.img_names = [f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if len(self.img_names) == 0:
            raise ValueError(f"No images found in {img_dir}. Please upload images first!")

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        img = Image.open(img_path).convert("RGB")
        return transform(img)

#Autoencoder Model
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1,output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1,output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 3, stride=2, padding=1,output_padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

#Training
def train_autoencoder(data_dir="data/unlabeled", epochs=10, lr=1e-3):
    try:
        dataset = ImageDataset(data_dir)
    except ValueError as e:
        print(e)
        print("Tip: Upload images to data/unlabeled/ using the Colab file uploader or download sample images.")
        return

    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    model = ConvAutoencoder().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    print(f"Training autoencoder on {len(dataset)} images...")

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            batch = batch.to(device)
            recon = model(batch)
            loss = criterion(recon, batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

    os.makedirs("models", exist_ok=True)
    torch.save(model.state_dict(),"models/autoencoder.pth")
    print("Autoencoder saved to models/autoencoder.pth")

if __name__ == "__main__":
    train_autoencoder()

Writing train_autoencoder.py


In [ ]:
%%writefile detect_anomaly.py
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as T
from train_autoencoder import ConvAutoencoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvAutoencoder().to(device)
model.load_state_dict(torch.load("models/autoencoder.pth", map_location=device))
model.eval()

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def detect_hazard(img_pil):
    """
    Returns: (is_anomalous: bool, recon_error: float, reconstructed_img: PIL)
    """
    img_tensor = transform(img_pil).unsqueeze(0).to(device)

    with torch.no_grad():
        recon = model(img_tensor)
        loss_fn = torch.nn.MSELoss()
        recon_error = loss_fn(recon, img_tensor).item()

    # Convert recon to PIL
    recon_pil = recon.cpu().squeeze(0)
    recon_pil = T.ToPILImage()(recon_pil)

    # Threshold (adjust based on validation)
    is_anomalous = recon_error > 0.02

    return is_anomalous, recon_error, recon_pil

Writing detect_anomaly.py


In [ ]:
%%writefile detect_helmet.py
from ultralytics import YOLO
from PIL import Image
import os

MODEL_PATH = "models/yolo_helmet.pt"

def download_pretrained_yolo():
    print("Downloading YOLOv8n for helmet detection...")
    model = YOLO("yolov8n.pt")
    model.save(MODEL_PATH)
    return model

def load_helmet_model():
    if not os.path.exists(MODEL_PATH):
        print("Model not found, downloading base YOLOv8...")
        download_pretrained_yolo()
    model = YOLO(MODEL_PATH)
    return model

def detect_helmet_in_image(img_pil, conf_threshold=0.5):
    model = load_helmet_model()
    results = model(img_pil, conf=conf_threshold)
    result = results[0]
    annotated_img = result.plot()
    annotated_pil = Image.fromarray(annotated_img)

    missing_helmets = 0
    detections = []

    for box in result.boxes:
        cls_id = int(box.cls)
        conf = float(box.conf)
        label = result.names[cls_id]
        detections.append(f"{label}: {conf:.2f}")
        if "head" in label.lower() or "person" in label.lower():
            missing_helmets += 1

    return annotated_pil, missing_helmets, ", ".join(detections)

Writing detect_helmet.py


In [ ]:
%%writefile generate_hazards.py
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os

os.makedirs("synthetic/generated_hazards", exist_ok=True)

def generate_unsafe_image(prompt="construction worker without hard hat, near crane, dangerous zone, realistic", num_images=1):
    pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
    pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

    images = pipe(prompt, num_images_per_prompt=num_images).images

    for i, img in enumerate(images):
        path = f"synthetic/generated_hazards/hazard_{i}.png"
        img.save(path)
        print(f"Generated: {path}")

    return images[0]

Writing generate_hazards.py


In [ ]:
%%writefile explain_alert.py
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.float16)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

def explain_hazard(helmet="No", zone="Danger", lighting="Poor"):
    prompt = f"""
    <|system|>
    You are a safety officer. Explain the hazard and recommend action.
    </s>
    <|user|>
    Worker helmet: {helmet}
    Location: {zone} zone
    Lighting: {lighting}
    Explain the risk clearly and suggest action.
    </s>
    <|assistant|>
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    outputs = model.generate(**inputs, max_new_tokens=200, do_sample=True, temperature=0.7)
    explanation = tokenizer.decode(outputs[0], skip_special_tokens=True)

    explanation = explanation.split("<|assistant|>")[-1].strip()
    return explanation

Writing explain_alert.py


In [ ]:
%%writefile app.py

import gradio as gr
from PIL import Image
import os

# Import modules
from detect_anomaly import detect_hazard
from generate_hazards import generate_unsafe_image
from explain_alert import explain_hazard
from detect_helmet import detect_helmet_in_image

last_result = {"anomalous": False, "error": 0.0}

def analyze_image(img):
    global last_result
    if img is None:
        return "Please upload an image", None, None

    pil_img = Image.fromarray(img).convert("RGB")
    is_anomalous, error, recon = detect_hazard(pil_img)
    last_result = {"anomalous": is_anomalous, "error": error}

    alert = "HAZARD DETECTED: No helmet or danger zone!" if is_anomalous else "No hazard detected."

    explanation = explain_hazard(
        helmet="No" if is_anomalous else "Yes",
        zone="Danger" if is_anomalous else "Safe",
        lighting="Normal"
    )

    return alert, recon, explanation

def create_synthetic():
    img = generate_unsafe_image()
    return img

#Gradio Interface
with gr.Blocks(title="AI Safety Inspector") as demo:
    gr.Markdown("# AI Safety Inspector\nDetects missing helmets, danger zones using **Unsupervised + Gen AI**")

    with gr.Tabs():
        with gr.Tab("Analyze Image"):
            with gr.Row():
                input_img = gr.Image(label="Upload Site Photo")
                output_recon = gr.Image(label="Reconstructed (Autoencoder)")
            output_alert = gr.Label(label="Status")
            output_explain = gr.Textbox(label="AI Safety Officer Says")
            btn = gr.Button("Analyze")
            btn.click(analyze_image, inputs=input_img, outputs=[output_alert, output_recon, output_explain])

        with gr.Tab("Helmet Detection (YOLOv8)"):
            gr.Markdown("Detects workers and checks if they are wearing helmets using YOLOv8.")

            with gr.Row():
                yolo_input = gr.Image(label="Upload Image")
                yolo_output_img = gr.Image(label="Detected Helmets")

            yolo_output_count = gr.Number(label="Workers Without Helmet")
            yolo_output_labels = gr.Textbox(label="Detections")

            yolo_btn = gr.Button("Run Helmet Detection")
            yolo_btn.click(
                detect_helmet_in_image,
                inputs=yolo_input,
                outputs=[yolo_output_img, yolo_output_count, yolo_output_labels]
            )

        with gr.Tab("Generate Synthetic Hazard (Gen AI)"):
            gen_output = gr.Image(label="Generated Unsafe Scenario")
            gen_btn = gr.Button("Generate No-Helmet Danger Scene")
            gen_btn.click(create_synthetic, outputs=gen_output)

        with gr.Tab("ℹ About"):
            gr.Markdown("""
            ### How It Works
            - Uses **autoencoder** to detect anomalies (no labels needed!)
            - **YOLOv8** detects helmets with bounding boxes
            - **Stable Diffusion** generates synthetic unsafe images
            - **TinyLlama** explains alerts in natural language
            - Runs on **Google Colab**
            """)

#Launch
demo.launch(share=True)

Writing app.py


In [ ]:
!mkdir -p models data/unlabeled synthetic/generated_hazards

In [ ]:
sample_urls = [
    "https://thumbs.dreamstime.com/b/construction-worker-without-helmet-building-site-unsafe-practice-216987654.jpg",
    "https://img.freepik.com/free-photo/worker-construction-site-without-helmet_23-2149174471.jpg",
    "https://ohsonline.com/articles/2014/07/01/-/media/OHS/OHS/Images/2014/07/rust.jpg",
    "https://www.alertmedia.com/wp-content/uploads/2021/09/OSHA_Blog-Image_A-1536x804.jpg",
    "https://www.citationcanada.com/app/uploads/2024/07/Addressing-Common-Excuses-for-Employees-Not-Wearing-PPE-at-Work-Feature.png"

]




import requests
from PIL import Image
from io import BytesIO
import time

print("Downloading sample construction images...")

for i, url in enumerate(sample_urls):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Check if it's actually an image
        content_type = response.headers.get('content-type', '')
        if 'image' not in content_type:
            print(f"Not an image: {url}")
            continue

        img = Image.open(BytesIO(response.content)).convert("RGB")
        img.save(f"data/unlabeled/sample_{i}.jpg")
        print(f" Downloaded: sample_{i}.jpg")
        time.sleep(0.5)  # Be nice to servers
    except Exception as e:
        print(f"Failed to download {url}: {str(e)[:100]}...")

print("Sample images added to data/unlabeled/")

 Downloaded: sample_0.jpg
 Downloaded: sample_1.jpg
 Downloaded: sample_2.jpg
 Downloaded: sample_3.jpg
 Downloaded: sample_4.jpg
Sample images added to data/unlabeled/


In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt -O models/yolo_helmet.pt

--2025-07-30 12:17:20--  https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/521807533/732c503e-9fcb-4a82-be7f-106baafbda15?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-30T13%3A17%3A51Z&rscd=attachment%3B+filename%3Dyolov8n.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-30T12%3A17%3A03Z&ske=2025-07-30T13%3A17%3A51Z&sks=b&skv=2018-11-09&sig=O1DI7a18NYBrQLwRztjwDk7VgeHWAlvQeVLxFj8573U%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1Mzg3ODE0MCwibmJmIjoxNzUzODc3ODQwLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvcmUud

In [ ]:
!python train_autoencoder.py

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100% 97.8M/97.8M [00:00<00:00, 180MB/s]
Training autoencoder on 5 images...
Epoch 1/10: 100% 1/1 [00:01<00:00,  1.13s/it]
Epoch 1, Loss: 1.8946
Epoch 2/10: 100% 1/1 [00:00<00:00, 23.13it/s]
Epoch 2, Loss: 1.8926
Epoch 3/10: 100% 1/1 [00:00<00:00, 25.89it/s]
Epoch 3, Loss: 1.8896
Epoch 4/10: 100% 1/1 [00:00<00:00, 26.05it/s]
Epoch 4, Loss: 1.8840
Epoch 5/10: 100% 1/1 [00:00<00:00, 25.62it/s]
Epoch 5, Loss: 1.8736
Epoch 6/10: 100% 1/1 [00:00<00:00, 27.36it/s]
Epoch 6, Loss: 1.8547
Epoch 7/10: 100% 1/1 [00:00<00:00, 26.42it/s]
Epoch 7, Loss: 1.8222
Epoch 8/10: 100% 1/1 [00:00<00:00, 27.21it/s]
Epoch 8, Loss: 1.7705
Epoch 9/10: 100% 1/1 [00:00<00:00, 26.87it/s]
Epoch 9, Loss: 1.6948
Epoch 10/10: 100% 1/1 [00:00<00:00, 27.38it/s]
Epoch 10, Loss: 1.5977
Autoencoder saved to models/autoencoder.pth


In [ ]:
!python app.py

2025-07-26 19:54:41.882860: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753559682.140216    1312 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753559682.211984    1312 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
tokenizer_config.json: 1.29kB [00:00, 8.91MB/s]
tokenizer.model: 100% 500k/500k [00:00<00:00, 13.6MB/s]
tokenizer.json: 1.84MB [00:00, 44.4MB/s]
special_tokens_map.json: 100% 551/551 [00:00<00:00, 4.44MB/s]
config.json: 100% 608/608 [00:00<00:00, 4.78MB/s]
model.safetensors: 100% 2.20G/2.20G [00:08<00:00, 254MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 693kB/s]
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultraly